# CNN Model

## Imports

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import json
from functions import train_model, load_data, evaluate_model

## CNN structure 

### Choices in this version:

1. Four convolutional layers helps learn more features from the images, but risks overfitting
2. Global average pooling pools across entire feature maps to one value but not locally, which although losing spatial information, may work sufficiently for classification and avoid overfitting
3.  Batch normalization helps standardize outputs of batches by normalizing, helps network run faster

In [ ]:
class MRI_CNN(nn.Module):
    '''This version of our CNN model includes Global average pooling, four convolutional layers, batch normalization
    '''
    def __init__(self, num_classes, dropout = 0.5):
        super(MRI_CNN, self).__init__()
        
        # 4 convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) 
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        
        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # Reduces spatial dimensions
        # Fully connected layer
        self.fc = nn.Linear(256, num_classes)
        
        # Dropout helps avoid overfitting
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool(x)

        x = self.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        
        x = self.global_avg_pool(x) 
        
        x = x.view(x.size(0), -1 #flatten before feeding to fc layer
        
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

In [ ]:
# run training on CNN image classifier
def run_cnn(
    batch_size, save_dir, dropout, lr, epochs
):
    NUM_CLASSES = 4

    train_loader, test_loader = load_data(batch_size)
    
    # instantiate the model
    model = MRI_CNN(num_classes=NUM_CLASSES, dropout=dropout)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # training
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    trained_model = train_model(save_dir, model, train_loader, device, optimizer, criterion, epochs)
    
    # run evaluation
    evaluate_model(trained_model, test_loader, device, save_dir)


In [ ]:
# run grid search
# Note: Change values below to be desired save path and search parameters
SAVE_PATH = "model_results/Cnn_"
DROPOUT = [0.1, 0.3, 0.5]
LR = [0.001, 0.0001]
EPOCHS = [10, 50]
BATCH_SIZE = [32]


COUNT = 1

for i in DROPOUT:
    for j in LR:
        for k in EPOCHS:
            for n in BATCH_SIZE:
                # make save directory if it does not exist
                SAVE_DIR = os.path.join(SAVE_PATH, str(COUNT))
                if not os.path.exists(SAVE_DIR):
                    os.makedirs(SAVE_DIR)
                # train and eval, save results
                run_cnn(n, SAVE_DIR, i, j, k)
                # save model configuration
                config = {
                    "dropout": i,
                    "learning_rate": j,
                    "epochs": k,
                    "batch_size": n,
                    "balanced": False,
                    "data_preprocessing": None
                }
                with open(os.path.join(SAVE_DIR, "config.json"), "w") as json_file:
                    json.dump(config, json_file, indent=4)
                COUNT+=1